In [12]:
!pip install flask_ngrok
from flask import Flask,render_template,redirect,url_for,request
from flask_ngrok import run_with_ngrok
import sqlite3
from IPython.display import HTML
import pandas as pd

In [ ]:
conn = sqlite3.connect('/content/drive/MyDrive/homo/database.db')
c=conn.cursor()

conn.execute("""CREATE TABLE IF NOT EXISTS homo (
                id INTEGER PRIMARY KEY,
                name TEXT,sex TEXT,age INTEGER)""")
                  
conn.commit()
conn.close()


app=Flask(__name__,template_folder='/content/drive/MyDrive/homo')
run_with_ngrok(app)


@app.route('/')
def home():
  return render_template('home.html')


@app.route('/enter')
def enter():
  return render_template('addrec.html')


@app.route('/addrec',methods=['POST'])
def addrec():
  try:
    nm=request.form['nm']
    sx=request.form['sx']
    ag=request.form['ag']
    conn=sqlite3.connect('/content/drive/MyDrive/homo/database.db')
    c=conn.cursor()
    c.execute("""INSERT  OR IGNORE INTO homo 
              (name,sex,age) VALUES (?,?,?)""",(nm,sx,ag))
    conn.commit()
    msg='Record succesfully added'

  except:
    conn.rollback()
    msg='Error in record insert'

  finally:
    conn.close()
    return render_template('result.html',msg=msg)

    
@app.route('/delete')
def delete():
  return render_template('delrec.html')


@app.route('/delrec',methods = ['POST'])  
def delrec():   
  try:
    id=request.form['id']
    conn=sqlite3.connect('/content/drive/MyDrive/homo/database.db') 
    c=conn.cursor()  
    c.execute("DELETE FROM homo WHERE id=?",id)
    conn.commit()  
    msg='Record successfully deleted'  
  except:  
    msg='Error in record delete'  
  finally: 
    conn.close() 
    return render_template("result.html",msg=msg)


@app.route('/view')
def view():
  conn=sqlite3.connect('/content/drive/MyDrive/homo/database.db')
  c=conn.cursor()
  df=pd.read_sql("SELECT * FROM homo",conn)
  title='People Info'
  return render_template('view.html',title=title,data=df.to_html(index=False))
  

if __name__=='__main__':
  app.run()

In [ ]:
HTML('/content/drive/MyDrive/templates/view.html')

In [ ]:
conn=sqlite3.connect('database.db')
c=conn.cursor()

pd.read_sql("SELECT * FROM homo",conn)